In [1]:
%pip install psycopg2 pandas

  Using cached psycopg2-2.9.10-cp312-cp312-linux_x86_64.whl
  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached numpy-2.1.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached numpy-2.1.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.0 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import psycopg2

# Establish the connection
conn = psycopg2.connect(
    dbname='postgres',
    user='postgres',
    password='postgres',
    host='localhost',
    port='5432'
)

# Create a cursor object
cursor = conn.cursor()

# Create the distributors table
cursor.execute('''
CREATE TABLE IF NOT EXISTS distributors (
    did SERIAL,
    dname TEXT,
    zipcode TEXT,
    CONSTRAINT distributors_pkey PRIMARY KEY (did)
);
''')
conn.commit()



# Insert initial data
cursor.execute("INSERT INTO distributors (did, dname, zipcode) VALUES (1, 'Old Distribution', '21201');")
cursor.execute("INSERT INTO distributors (did, dname, zipcode) VALUES (2, 'New Distribution', '12345');")
cursor.execute("INSERT INTO distributors (did, dname, zipcode) VALUES (3, 'Another Distributor', '21201');")
conn.commit()


# First SQL statement
try:
    cursor.execute('''
    INSERT INTO distributors AS d (did, dname) VALUES (8, 'Anvil Distribution')
        ON CONFLICT (did) DO UPDATE
        SET dname = EXCLUDED.dname || ' (formerly ' || d.dname || ')'
        WHERE d.zipcode <> '21201';
    ''')
    conn.commit()
except Exception as e:
    print(f"Error: {e}")

try:
    cursor.execute('''
    INSERT INTO distributors AS d (did, dname) VALUES (8, 'Anvil Distribution')
        ON CONFLICT (did) DO UPDATE
        SET dname = EXCLUDED.dname || ' (formerly ' || d.dname || ')';
    ''')
    conn.commit()
except Exception as e:
    print(f"Error: {e}")

# Second SQL statement
try:
    cursor.execute('''
    INSERT INTO distributors (did, dname) VALUES (9, 'Antwerp Design')
        ON CONFLICT ON CONSTRAINT distributors_pkey DO NOTHING;
    ''')
    conn.commit()
except Exception as e:
    print(f"Error: {e}")

try:
    cursor.execute('''
    INSERT INTO distributors (did, dname) VALUES (9, 'Antwerp Design')
        ON CONFLICT ON CONSTRAINT distributors_pkey DO NOTHING;
    ''')
    conn.commit()
except Exception as e:
    print(f"Error: {e}")



# Query the table
cursor.execute("SELECT * FROM distributors;")
rows = cursor.fetchall()

for row in rows:
    print(row)


# Close the cursor and connection
cursor.close()
conn.close()


(1, 'Old Distribution', '21201')
(2, 'New Distribution', '12345')
(3, 'Another Distributor', '21201')
(8, 'Anvil Distribution (formerly Anvil Distribution)', None)
(9, 'Antwerp Design', None)
